<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/2-Advance/FinRL_Ensemble_StockTrading_ICAIF_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install all the packages through FinRL library


In [14]:
# ## install finrl library
!pip install wrds
!pip install swig
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install pandas_market_calendars


zsh:1: command not found: apt-get
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-28tnnfnd
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-28tnnfnd
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 69776b349ee4e63efe3826f318aef8e5c5f59648
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-cuh9bhwl/elegantrl_b0c8be6984e34e1f89cfec0ad9a9d653
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-cuh9bhwl/elegantrl_b

## Import Packages

In [15]:
# ===========================
# Suppress Warnings
# ===========================
import warnings
warnings.filterwarnings("ignore")

# ===========================
# Standard Libraries
# ===========================
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# matplotlib.use('Agg')  
import shutil
import glob


# ===========================
# Enable Inline Plotting (Jupyter)
# ===========================
%matplotlib inline

# ===========================
# FinRL Imports
# ===========================
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.main import check_and_make_directories
from finrl.config import (
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
)

# ===========================
# Create Necessary Directories
# ===========================
check_and_make_directories([
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR
])


# ===========================
# Custom Imports (model.py)
# ===========================
sys.path.append(os.path.abspath("."))  
from models import DRLEnsembleAgent

sys.path.append("../FinRL-Library")


## `process_csv_to_features(csv_path)`

Processes financial data from a CSV by adding technical indicators and turbulence features.

### **Parameters**
- `csv_path` *(str)*: Path to the raw financial data CSV.

### **Workflow**
1. Load data.
2. Identify 5-day and 7-day tickers.
3. Apply technical indicators.
4. Combine datasets.
5. Add turbulence feature.
6. Clean `NaN` and infinite values.

### **Returns**
- `processed` *(DataFrame)*: Feature-enhanced, cleaned DataFrame for modeling.


In [16]:
def process_csv_to_features(csv_path):
    # Step 1: Load Data
    df = pd.read_csv(csv_path)

    # Step 2: Identify 5-day and 7-day tickers
    day_values_per_tic = df.groupby('tic')['day'].apply(lambda x: sorted(x.unique())).reset_index()
    day_values_per_tic.columns = ['tic', 'unique_days']

    tics_5day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(5)))]['tic']
    tics_7day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(7)))]['tic']

    df_5day_full = df[df['tic'].isin(tics_5day)]
    df_7day_full = df[df['tic'].isin(tics_7day)]

    # Step 3: Apply Technical Indicators
    fe_ti = FeatureEngineer(
        use_technical_indicator=True,
        use_turbulence=False,
        user_defined_feature=False
    )
    df_5day_full = fe_ti.preprocess_data(df_5day_full)
    if not df_7day_full.empty:
        df_7day_full = fe_ti.preprocess_data(df_7day_full)
    else:
        print("[Info] df_7day_full is empty. Skipping technical indicators.")

    # Step 4: Combine and Clean Index
    combined_df = pd.concat([df_5day_full, df_7day_full], ignore_index=False)
    combined_df.index = range(len(combined_df))

    # Step 5: Remove dates with only one ticker
    combined_df['date'] = pd.to_datetime(combined_df['date'])
    combined_df = combined_df[combined_df.groupby('date')['date'].transform('count') > 1]
    combined_df = combined_df.sort_values(['date', 'tic']).reset_index(drop=True)

    # Step 6: Apply Turbulence Feature
    fe_turb = FeatureEngineer(
        use_technical_indicator=False,
        use_turbulence=True,
        user_defined_feature=False
    )
    processed = fe_turb.preprocess_data(combined_df)

    # Step 7: Final Cleaning
    processed = processed.copy()
    processed = processed.fillna(0)
    processed = processed.replace(np.inf, 0)

    return processed


## `setup_drl_ensemble_agent(...)`

Initializes a `DRLEnsembleAgent` with dynamic environment and parameter settings.

### **Key Parameters**
- `processed_df` *(DataFrame)*: Data with features for training and trading.
- `indicators` *(list)*: Technical indicators used.
- `train_start_date`, `train_end_date`: Training period.
- `trade_start_date`, `trade_end_date`: Trading period.
- `rebalance_window`, `validation_window`: Rebalancing and validation frequency.
- `initial_amount`, `transaction_cost`, `hmax`, `reward_scaling`: Trading environment settings.

### **Returns**
- `agent`: Configured `DRLEnsembleAgent` ready for training and trading.


In [17]:
def setup_drl_ensemble_agent(processed_df,  
                              indicators, 
                              train_start_date, 
                              train_end_date,
                              trade_start_date, 
                              trade_end_date, 
                              rebalance_window=63, 
                              validation_window=63, 
                              initial_amount=1_000_000,
                              transaction_cost=0.001,
                              hmax=100,
                              reward_scaling=1e-4,
                              print_verbosity=5):
    """
    Setup DRLEnsembleAgent with flexible date and parameter configuration.
    """

    # 1. Calculate dynamic parameters
    stock_dimension = len(processed_df.tic.unique())
    state_space = 1 + 2 * stock_dimension + len(indicators) * stock_dimension

    # 2. Environment configuration
    env_kwargs = {
        "hmax": hmax,
        "initial_amount": initial_amount,
        "buy_cost_pct": transaction_cost,
        "sell_cost_pct": transaction_cost,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": reward_scaling,
        "print_verbosity": print_verbosity
    }

    # 3. Initialize DRLEnsembleAgent
    agent = DRLEnsembleAgent(
        df=processed_df,
        train_period=(train_start_date, train_end_date),
        val_test_period=(trade_start_date, trade_end_date),
        rebalance_window=rebalance_window,
        validation_window=validation_window,
        **env_kwargs
    )

    return agent


## DRL Model Hyperparameters & Training Timesteps

Defines hyperparameters for five DRL algorithms and their training timesteps.

### **Model Hyperparameters**
- **A2C**:  
  `n_steps`, `ent_coef`, `learning_rate`

- **PPO**:  
  `n_steps`, `ent_coef`, `learning_rate`, `batch_size`

- **DDPG**:  
  `buffer_size`, `learning_rate`, `batch_size`

- **SAC**:  
  `batch_size`, `buffer_size`, `learning_rate`, `learning_starts`, `ent_coef`

- **TD3**:  
  `batch_size`, `buffer_size`, `learning_rate`

### **Training Timesteps**
- Each model: `10,000` timesteps  
  *(Defined in `timesteps_dict`)*



In [18]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.005
                    }

PPO_model_kwargs = {
                    "ent_coef":0.005,
                    "n_steps": 2048,
                    "learning_rate": 0.00001,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      "buffer_size": 50_000,
                      "learning_rate": 0.005,
                      "batch_size": 128
                    }

SAC_model_kwargs = {
                      "batch_size": 128,
                      "buffer_size": 100000,
                      "learning_rate": 0.005,
                      "learning_starts": 100,
                      "ent_coef": 0.1,
                    }

TD3_model_kwargs = {
                      "batch_size": 100,
                      "buffer_size": 1000000,
                      "learning_rate": 0.001
                   }


timesteps_dict = {'a2c' : 30_000,
                 'ppo' : 30_000,
                 'ddpg' : 30_000,
                  'sac' : 30_000,
                 'td3' : 30_000,
                 }

## `run_ensemble_and_generate_daily_return(...)`

Executes a DRL Ensemble Strategy, tracks portfolio performance, calculates daily returns, and organizes output files.

### **Key Features**
- Runs `ensemble_agent` with specified model hyperparameters and timesteps.
- Tracks continuous portfolio value across rebalancing periods.
- Generates and saves a portfolio value plot.
- Calculates daily returns and exports to CSV.
- Organizes trained models, logs, and results into a structured folder.

### **Parameters**
- `ensemble_agent`: Initialized DRLEnsembleAgent.
- Model kwargs: `A2C_kwargs`, `PPO_kwargs`, `DDPG_kwargs`, `SAC_kwargs`, `TD3_kwargs`.
- `timesteps_dict`: Training timesteps per model.
- `processed_df`: Feature-enhanced DataFrame.
- Date ranges, rebalancing configs, and file management options.

### **Returns**
- `df_daily_return` *(DataFrame)*: Daily return series for the portfolio.


In [19]:
def run_ensemble_and_generate_daily_return(ensemble_agent, 
                                            A2C_kwargs, PPO_kwargs, DDPG_kwargs, SAC_kwargs, TD3_kwargs, 
                                            timesteps_dict, 
                                            processed_df, 
                                            trade_start_date, trade_end_date, 
                                            rebalance_window, validation_window, 
                                            output_csv_name="df_daily_return.csv",
                                            initial_fund=1_000_000,
                                            original_csv_path="data.csv"):
    """
    Runs DRL Ensemble Strategy, tracks continuous portfolio value, 
    calculates daily returns, saves outputs, and organizes files into a folder.
    """

    # ===========================
    # Create Necessary Directories
    # ===========================
    check_and_make_directories([
        TRAINED_MODEL_DIR,
        TENSORBOARD_LOG_DIR,
        RESULTS_DIR
    ])


    # === Step 1: Create Folder Based on CSV Name ===
    base_name = os.path.splitext(os.path.basename(original_csv_path))[0]
    target_folder = f"{base_name}"
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"[INFO] Created folder: {target_folder}")

    # === Step 2: Run Ensemble Strategy ===
    print("[INFO] Running Ensemble Strategy...")
    df_summary = ensemble_agent.run_ensemble_strategy(
        A2C_kwargs, PPO_kwargs, DDPG_kwargs, SAC_kwargs, TD3_kwargs, timesteps_dict
    )

    # === Step 3: Prepare Trade Dates ===
    unique_trade_date = processed_df[
        (processed_df.date >= trade_start_date) & (processed_df.date <= trade_end_date)
    ].date.unique()

    current_value = initial_fund
    portfolio_tracking = []
    is_first_file = True

    rebalance_points = list(range(rebalance_window + validation_window, len(unique_trade_date) + 1, rebalance_window))

    # === Step 4: Track Portfolio Value Across Rebalances ===
    for i in rebalance_points:
        file_path = f'results/account_value_trade_ensemble_{i}.csv'
        if os.path.exists(file_path):
            temp = pd.read_csv(file_path)

            if is_first_file:
                first_date = temp.loc[0, 'date']
                original_value = temp.loc[0, 'account_value']
                portfolio_tracking.append({
                    'date': first_date,
                    'portfolio_value': current_value,
                    'original_account_value': original_value
                })
                start_idx = 1
                is_first_file = False
            else:
                start_idx = 1

            for idx in range(start_idx, len(temp)):
                daily_return = temp.loc[idx, 'daily_return']
                date = temp.loc[idx, 'date']
                original_value = temp.loc[idx, 'account_value']
                if pd.notna(daily_return):
                    current_value *= (1 + daily_return)
                    portfolio_tracking.append({
                        'date': date,
                        'portfolio_value': current_value,
                        'original_account_value': original_value
                    })
        else:
            print(f"[Warning] File does not exist: {file_path}")

    df_portfolio = pd.DataFrame(portfolio_tracking)

    # === Step 5: Plot Portfolio Value ===
    plt.figure(figsize=(14,6))
    plt.plot(pd.to_datetime(df_portfolio['date']), df_portfolio['portfolio_value'], label='Continuous Portfolio Value')
    plt.plot(pd.to_datetime(df_portfolio['date']), df_portfolio['original_account_value'], label='Original (Resetting) Account Value', linestyle='--')
    plt.title('Portfolio Value: Continuous vs Original')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plot_path = os.path.join(target_folder, "portfolio_value_plot.png")
    plt.savefig(plot_path)
    print(f"[INFO] Portfolio value plot saved to: {plot_path}")
    plt.close()

    # === Step 6: Calculate Daily Returns ===
    df_daily_return = df_portfolio.copy()
    df_daily_return["daily_return"] = df_daily_return["portfolio_value"].pct_change()
    df_daily_return = df_daily_return.infer_objects(copy=False)
    df_daily_return.loc[0, "daily_return"] = 0.0
    df_daily_return = df_daily_return[["date", "daily_return"]]

    # === Step 7: Save Daily Return CSV into Folder ===
    csv_full_path = os.path.join(target_folder, output_csv_name)
    df_daily_return.to_csv(csv_full_path, index=False)
    print(f"[INFO] Daily return saved to: {csv_full_path}")
    
    # === Step 8: Merge Trade Action Files ===
    print("[INFO] Merging trade action files...")
    action_files = sorted(glob.glob(os.path.join(RESULTS_DIR, "actions_trade_ensemble_*.csv")))

    if action_files:
        df_actions_list = [pd.read_csv(f) for f in action_files]
        df_actions_merged = pd.concat(df_actions_list, ignore_index=True)

        # Save merged actions
        actions_output_path = os.path.join(target_folder, "merged_trade_actions.csv")
        df_actions_merged.to_csv(actions_output_path, index=False)
        print(f"[INFO] Merged trade actions saved to: {actions_output_path}")

        # Optional: Basic Trade Stats
        total_trades = (df_actions_merged.drop(columns=['date'], errors='ignore') != 0).sum().sum()
        print(f"[INFO] Total trades executed: {total_trades}")

    else:
        print("[Warning] No trade action files found to merge.")

    # === Step 9: Move Directories into the Folder ===
    dirs_to_move = [TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR]

    for dir_path in dirs_to_move:
        if os.path.exists(dir_path):
            dest_path = os.path.join(target_folder, os.path.basename(dir_path))
            if os.path.exists(dest_path):
                shutil.rmtree(dest_path)  
            shutil.move(dir_path, target_folder)
            print(f"[INFO] Moved {dir_path} to {target_folder}/")
        else:
            print(f"[Warning] Directory not found: {dir_path}")

    return df_daily_return


## DRL Ensemble Strategy Workflow

This part processes datasets, initializes DRL ensemble agents, runs the ensemble trading strategy, and generates daily returns for three different datasets.

### **Workflow Overview**
For each dataset:
1. **Process Data**  
   Apply feature engineering using `process_csv_to_features()`.

2. **Setup DRL Ensemble Agent**  
   Configure the agent with `setup_drl_ensemble_agent()`.

3. **Run Ensemble Strategy & Generate Daily Returns**  
   Execute `run_ensemble_and_generate_daily_return()` to:
   - Train models (A2C, PPO, DDPG, SAC, TD3)
   - Track portfolio value
   - Calculate and save daily returns
   - Organize outputs

---

### **Datasets Processed**
1. `2007-2025_no_crypto.csv`  
   - **Train**: 2007-06-01 to 2023-01-03  
   - **Trade**: 2023-01-04 to 2025-04-11  

2. `2015-2025_crypto.csv`  
   - **Train**: 2015-02-02 to 2023-01-03  
   - **Trade**: 2023-01-04 to 2025-04-11  

3. `2015-2025_no_crypto.csv`  
   - **Train**: 2015-02-02 to 2023-01-03  
   - **Trade**: 2023-01-04 to 2025-04-11  

---

### **Outputs**
- Daily return CSV: `df_daily_return_ensemble.csv` (saved in dataset-specific folders)
- Portfolio value plots
- Organized directories for models, logs, and results


In [20]:
processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')

ensemble_agent_0 = setup_drl_ensemble_agent(
    processed_df = processed_0,
    indicators = INDICATORS,
    train_start_date = '2007-06-01',
    train_end_date = '2023-01-03',
    trade_start_date = '2023-01-04',
    trade_end_date = '2025-04-11',
    rebalance_window = 63,
    validation_window = 63
)

df_daily_return = run_ensemble_and_generate_daily_return(
    ensemble_agent=ensemble_agent_0,
    A2C_kwargs=A2C_model_kwargs,
    PPO_kwargs=PPO_model_kwargs,
    DDPG_kwargs=DDPG_model_kwargs,
    SAC_kwargs=SAC_model_kwargs,
    TD3_kwargs=TD3_model_kwargs,
    timesteps_dict=timesteps_dict,
    processed_df=processed_0,
    trade_start_date='2023-01-04',
    trade_end_date='2025-04-11',
    rebalance_window=63,
    validation_window=63,
    output_csv_name="df_daily_return_ensemble.csv",
    initial_fund=1_000_000,
    original_csv_path="2007-2025_no_crypto.csv"
)

Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.
Successfully added turbulence index
[INIT] Unique trade dates from 2023-01-04 to 2025-04-11
[INIT] Total trading days: 570
[INIT] First 5 dates: <DatetimeArray>
['2023-01-04 00:00:00', '2023-01-05 00:00:00', '2023-01-06 00:00:00',
 '2023-01-09 00:00:00', '2023-01-10 00:00:00']
Length: 5, dtype: datetime64[ns]
[INIT] Last 5 dates: <DatetimeArray>
['2025-04-07 00:00:00', '2025-04-08 00:00:00', '2025-04-09 00:00:00',
 '2025-04-10 00:00:00', '2025-04-11 00:00:00']
Length: 5, dtype: datetime64[ns]

[INIT PREVIEW] ===== Rebalancing Schedule Summary =====
Iter     Train End    Val Start    Val End      Trade Start  Trade End   
126      2023-01-04 00:00:00 2023-01-04 00:00:00 2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00
189      2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00 2023-07-07 00:00:00 2023-10-05 00:00:00
252      2023-07-07 00:00:00 2023-07-07 00:00:00 202

In [21]:
processed_1 = process_csv_to_features('2015-2025_crypto.csv')

ensemble_agent_1 = setup_drl_ensemble_agent(
    processed_df = processed_1,
    indicators = INDICATORS,
    train_start_date = '2015-02-02',
    train_end_date = '2023-01-03',
    trade_start_date = '2023-01-04',
    trade_end_date = '2025-04-11',
    rebalance_window = 63,
    validation_window = 63
)

df_daily_return = run_ensemble_and_generate_daily_return(
    ensemble_agent=ensemble_agent_1,
    A2C_kwargs=A2C_model_kwargs,
    PPO_kwargs=PPO_model_kwargs,
    DDPG_kwargs=DDPG_model_kwargs,
    SAC_kwargs=SAC_model_kwargs,
    TD3_kwargs=TD3_model_kwargs,
    timesteps_dict=timesteps_dict,
    processed_df=processed_1,
    trade_start_date='2023-01-04',
    trade_end_date='2025-04-11',
    rebalance_window=63,
    validation_window=63,
    output_csv_name="df_daily_return_ensemble.csv",
    initial_fund=1_000_000,
    original_csv_path="2015-2025_crypto.csv"
)

Successfully added technical indicators
Successfully added technical indicators
Successfully added turbulence index
[INIT] Unique trade dates from 2023-01-04 to 2025-04-11
[INIT] Total trading days: 570
[INIT] First 5 dates: <DatetimeArray>
['2023-01-04 00:00:00', '2023-01-05 00:00:00', '2023-01-06 00:00:00',
 '2023-01-09 00:00:00', '2023-01-10 00:00:00']
Length: 5, dtype: datetime64[ns]
[INIT] Last 5 dates: <DatetimeArray>
['2025-04-07 00:00:00', '2025-04-08 00:00:00', '2025-04-09 00:00:00',
 '2025-04-10 00:00:00', '2025-04-11 00:00:00']
Length: 5, dtype: datetime64[ns]

[INIT PREVIEW] ===== Rebalancing Schedule Summary =====
Iter     Train End    Val Start    Val End      Trade Start  Trade End   
126      2023-01-04 00:00:00 2023-01-04 00:00:00 2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00
189      2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00 2023-07-07 00:00:00 2023-10-05 00:00:00
252      2023-07-07 00:00:00 2023-07-07 00:00:00 2023-10-05 00:00:00 2023

In [22]:
processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')

ensemble_agent_2 = setup_drl_ensemble_agent(
    processed_df = processed_2,
    indicators = INDICATORS,
    train_start_date = '2015-02-02',
    train_end_date = '2023-01-03',
    trade_start_date = '2023-01-04',
    trade_end_date = '2025-04-11',
    rebalance_window = 63,
    validation_window = 63
)

df_daily_return = run_ensemble_and_generate_daily_return(
    ensemble_agent=ensemble_agent_2,
    A2C_kwargs=A2C_model_kwargs,
    PPO_kwargs=PPO_model_kwargs,
    DDPG_kwargs=DDPG_model_kwargs,
    SAC_kwargs=SAC_model_kwargs,
    TD3_kwargs=TD3_model_kwargs,
    timesteps_dict=timesteps_dict,
    processed_df=processed_2,
    trade_start_date='2023-01-04',
    trade_end_date='2025-04-11',
    rebalance_window=63,
    validation_window=63,
    output_csv_name="df_daily_return_ensemble.csv",
    initial_fund=1_000_000,
    original_csv_path="2015-2025_no_crypto.csv"
)

Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.
Successfully added turbulence index
[INIT] Unique trade dates from 2023-01-04 to 2025-04-11
[INIT] Total trading days: 570
[INIT] First 5 dates: <DatetimeArray>
['2023-01-04 00:00:00', '2023-01-05 00:00:00', '2023-01-06 00:00:00',
 '2023-01-09 00:00:00', '2023-01-10 00:00:00']
Length: 5, dtype: datetime64[ns]
[INIT] Last 5 dates: <DatetimeArray>
['2025-04-07 00:00:00', '2025-04-08 00:00:00', '2025-04-09 00:00:00',
 '2025-04-10 00:00:00', '2025-04-11 00:00:00']
Length: 5, dtype: datetime64[ns]

[INIT PREVIEW] ===== Rebalancing Schedule Summary =====
Iter     Train End    Val Start    Val End      Trade Start  Trade End   
126      2023-01-04 00:00:00 2023-01-04 00:00:00 2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00
189      2023-04-05 00:00:00 2023-04-05 00:00:00 2023-07-07 00:00:00 2023-07-07 00:00:00 2023-10-05 00:00:00
252      2023-07-07 00:00:00 2023-07-07 00:00:00 202